In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=c9a3e9ee073915a8802f85f1bab2d6c9a5f7da805f6508b4f8c114123c9d12e9
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# criando uma sessão spark
spark = (SparkSession.builder
                     .master('local')
                     .appName('ex2 13-04-23')
                     .config('spark.ui.port', '4050')
                     .getOrCreate()
)

In [ ]:
# abrindo o dataframe no spark
df = (
    spark.read
         .format('csv')
         .option('delimiter', ',')
         .option('header', 'true')
         .option('inferschema', 'true')
         .option("encoding", "utf-8")
         .option('escape' , '"')
         .load('/content/drive/MyDrive/PySpark/spotify - spotify.csv')
)

In [ ]:
#primeiro passo é descobrir o tamanho do dataframe
print(f'({df.count()}, {len(df.columns)})')

(18835, 16)


In [ ]:
#segundo passo é verificar a tipagem das colunas
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- song_name: string (nullable = true)
 |-- song_popularity: string (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
# visualizar todo o dataframe
df.show(truncate=False)

+---+--------------------------+---------------+----------------+----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|_c0|song_name                 |song_popularity|song_duration_ms|acousticness          |danceability           |energy               |instrumentalness     |key|liveness              |loudness               |audio_mode|speechiness           |tempo  |time_signature|audio_valence        |
+---+--------------------------+---------------+----------------+----------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+----------------------+-------+--------------+---------------------+
|0  |Boulevard of Broken Dreams|73             |262333          |0.005520000000000001kg|0.496mol/L             |682                  |2.94e

In [ ]:
# terceiro passo e saber se existem colunas duplicadas
# para isto drop na coluna ID, somente se não foi fazer um join de outros dataframes
df = df.drop(F.col('_c0'))

In [ ]:
# criar primeiro backup sempre antes de qualquer drop
df_backup1 = df

In [ ]:
# contar duplicados
total = df.count() - df.dropDuplicates().count()
print(total)

3903


In [ ]:
# realizar backup sempre antes de um drop
df_backup2 = df

In [ ]:
# dropar as colunas duplicadas
df = df.drop_duplicates()

In [ ]:
# verificar a contagem para verificar se foi aplicado o drop corretamente
df.count()

14932

In [ ]:
# verificar summary de todo o data frame
df.summary().show()

+-------+--------------------+------------------+--------------------+------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+
|summary|           song_name|   song_popularity|    song_duration_ms|acousticness|danceability|            energy|  instrumentalness|               key|          liveness|          loudness|        audio_mode|       speechiness|             tempo|      time_signature|     audio_valence|
+-------+--------------------+------------------+--------------------+------------+------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+
|  count|               14932|             14932|               14932|       14932|       14932|             14932|             14932

In [ ]:
# limpeza dos dados com informações inconsistentes na coluna acoustincness
# primeiro tentando por replace
df.replace('kg', '', subset='acousticness').show()

+--------------------+---------------+----------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|              energy|    instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|             0.211kg|0.755999999999999...|                 0.8|                 723|  5|                  47|              -5.158|         1|      

In [ ]:
# replace não deu certo, passar para o próximo metodo
# podemos tentar o método split
df.withColumn('acousticness', F.split(F.col('acousticness'), 'kg').getItem(0)).show()

+--------------------+---------------+----------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|        danceability|              energy|    instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+--------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|               0.211|0.755999999999999...|                 0.8|                 723|  5|                  47|              -5.158|         1|      

In [ ]:
# outra opção que funciona é o regax replace
df.select(F.regexp_replace(F.col("acousticness"), 'kg', '')).show()

+-------------------------------------+
|regexp_replace(acousticness, kg, , 1)|
+-------------------------------------+
|                                0.211|
|                               0.0767|
|                                0.188|
|                                0.159|
|                                0.236|
|                                0.232|
|                                0.405|
|                                0.317|
|                               0.0826|
|                                 0.13|
|                 0.002610000000000...|
|                               0.0173|
|                                0.431|
|                                0.688|
|                                0.206|
|                                0.899|
|                                0.149|
|                               0.0232|
|                                0.104|
|                   0.8059999999999999|
+-------------------------------------+
only showing top 20 rows



In [ ]:
# backup para aplicar a alteração na coluna acoustincness
df_backup3 = df

In [ ]:
# modificando a coluna acousticness usando o split
df = df.withColumn('acousticness', F.split(F.col('acousticness'), 'kg').getItem(0))

In [ ]:
# verificando a alteração
df.show(truncate=False)

+----------------------------------------+---------------+----------------+---------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|song_name                               |song_popularity|song_duration_ms|acousticness         |danceability           |energy               |instrumentalness     |key|liveness              |loudness               |audio_mode|speechiness|tempo                 |time_signature|audio_valence         |
+----------------------------------------+---------------+----------------+---------------------+-----------------------+---------------------+---------------------+---+----------------------+-----------------------+----------+-----------+----------------------+--------------+----------------------+
|Lambada - Original Version 1989         |67             |207466          |0.211                |

In [ ]:
# modificando a coluna danceability usando o split
df = df.withColumn('danceability', F.split(F.col('danceability'), 'mol/L').getItem(0))

In [ ]:
# o ideal e aplicar o summary em todas as colunas numericas
df.select('song_popularity').summary().show()

+-------+------------------+
|summary|   song_popularity|
+-------+------------------+
|  count|             14932|
|   mean| 48.75768535262206|
| stddev|20.379617796732816|
|    min|                 0|
|    25%|              37.0|
|    50%|              52.0|
|    75%|              64.0|
|    max|           nao_sei|
+-------+------------------+



In [ ]:
# verificar a quantidade de valores nulos
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            1|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
# verificar quantidade de nao_sei em todas as colunas
df.filter(F.col('song_popularity').contains('nao')).show()

+-------------------+---------------+--------------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|          song_name|song_popularity|    song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+-------------------+---------------+--------------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|Party In The U.S.A.|        nao_sei|0.8220000000000001kg|  0.519mol/L|        0.36|   0.0|              10|177|  -8.575|       0|       105|      97.42|    4|           0.7|         null|
+-------------------+---------------+--------------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
# drop as colunas com não sei
# da replace de todos não sei para None valor nulo
# para depois da drop em tudos eles usando o how= any (  se a linha tiver um valor ja drop ela, o all= se todos os valores tiverem nulos)
df = df.replace('nao_sei', None)
df = df.replace('0.nao_sei', None)
df = df.dropna(how='any')

In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|      danceability|              energy|    instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+------------------+--------------------+--------------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|               0.211|0.7559999999999999|                 0.8|                 723|  5|                  47|              -5.158|         1|        329|  

In [ ]:
# tratar a coluna instrumentalness
# usar summary para entender qual a faixa de valor e normal (0 a 1)
df.select(F.col('instrumentalness')).summary().show()

+-------+------------------+
|summary|  instrumentalness|
+-------+------------------+
|  count|             14919|
|   mean|164.09098902340065|
| stddev|275.64426920379583|
|    min|               0.0|
|    25%|               0.0|
|    50%|           1.14E-5|
|    75%|             223.0|
|    max|               999|
+-------+------------------+



In [ ]:
# analisar como os valores desta coluna estao organizados
df.select(F.col('instrumentalness')).orderBy(F.col('instrumentalness').desc()).show()

+----------------+
|instrumentalness|
+----------------+
|             999|
|             998|
|             998|
|             998|
|             997|
|             997|
|             996|
|             995|
|             994|
|             994|
|             994|
|             992|
|             992|
|             991|
|             991|
|             991|
|             991|
|              99|
|              99|
|              99|
+----------------+
only showing top 20 rows



In [ ]:
# retirar as stings da coluna instrumentalness
from pyspark.sql.functions import regexp_extract
df = df.withColumn("instrumentalness", regexp_extract("instrumentalness", r"\d+", 0))


In [ ]:
from pyspark.sql.types import IntegerType, FloatType, LongType, DecimalType

In [ ]:
# converter cada coluna para a tipagem certa, vou fazer uma por uma porque vi que algumas estavam dando problemas com dados nulos
df = df.withColumn('song_popularity', F.col('song_popularity').cast(IntegerType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
# verificar a tipagem para transformar
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: integer (nullable = true)
 |-- song_duration_ms: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)



In [ ]:
backup_song_duration_m = df
df = df.withColumn("song_duration_ms", regexp_extract("song_duration_ms", r"\d+", 0))
df = df.withColumn('song_duration_ms', F.col('song_duration_ms').cast(IntegerType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+

+--------------------+---------------+----------------+--------------------+------------------+--------------------+--------

In [ ]:
# criar uma segunda coluna de song duration em segundos e dropar a antiga
df = df.withColumn('song_duration', F.round(F.col('song_duration_ms') / 60000, 2))
df = df.drop(F.col('song_duration_ms'))

In [ ]:
df.show()

+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|        acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|               0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|        

In [ ]:
# primeira tentativa converter para INTERGERTYPE
backup_acousticness = df
df = df.withColumn("acousticness", regexp_extract("acousticness", r"\d+", 0))
df = df.withColumn('acousticness', F.col('acousticness').cast(IntegerType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|song_name|song_popularity|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|        0|              0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|            1|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+------

In [ ]:
df = backup_acousticness

In [ ]:
df.show()

+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|        acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|               0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|        

In [ ]:
# segunda tentativa converter para FLOATTYPE
backup_acousticness = df
df = df.withColumn("acousticness", regexp_extract("acousticness", r"\d+", 0))
df = df.withColumn('acousticness', F.col('acousticness').cast(FloatType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|song_name|song_popularity|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|        0|              0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|            1|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+------

In [ ]:
df = backup_acousticness

In [ ]:
df.show()

+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|        acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|               0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|        

In [ ]:
# terceira tentativa  ARRENDONDAR E CONVERTER PARA FLOATTYPE
from pyspark.sql.functions import round

df = df.withColumn("acousticness", round(df["acousticness"], 4))

In [ ]:
df.show()

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|           0|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|                 967|         3.46|
|  L

In [ ]:
# terceira tentativa  ARRENDONDAR E CONVERTER PARA FLOATTYPE
backup_acousticness = df
df = df.withColumn("acousticness", regexp_extract("acousticness", r"\d+", 0))
df = df.withColumn('acousticness', F.col('acousticness').cast(FloatType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|song_name|song_popularity|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|        0|              0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|            1|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+------

In [ ]:
df = backup_acousticness
df.show()

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|       0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|                 967|         3.46|
|  L

In [ ]:
# quinta tentativa  ARRENDONDAR E CONVERTER PARA DECIMALTYPE
backup_acousticness = df
df = df.withColumn("acousticness", regexp_extract("acousticness", r"\d+", 0))
df = df.withColumn('acousticness', F.col('acousticness').cast(DecimalType(10, 4)))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|song_name|song_popularity|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|        0|              0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|            1|
+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+------

In [ ]:
 df = backup_acousticness

In [ ]:
df.printSchema()

root
 |-- song_name: string (nullable = true)
 |-- song_popularity: integer (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- audio_mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- time_signature: string (nullable = true)
 |-- audio_valence: string (nullable = true)
 |-- song_duration: double (nullable = true)



In [ ]:
df.show()

+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|           song_name|song_popularity|acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|song_duration|
+--------------------+---------------+------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+-------------+
|Lambada - Origina...|             67|           0|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|                 967|         3.46|
|  L

In [ ]:
df = backup_song_duration_m
df.show()

+--------------------+---------------+----------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|               0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.9

In [ ]:
backup_danceability = df
# df = df.withColumn("danceabilitys", regexp_extract("danceabilitys", r"\d+", 0))
df = df.withColumn('danceability', F.col('danceability').cast(FloatType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()
df.show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+

+--------------------+---------------+----------------+--------------------+------------+--------------------+--------------

In [ ]:
# tentei converter para Intertype a coluna energy mas nao deu certo
backup_energy = df
df = df.withColumn("energys", regexp_extract("energy", r"\d+", 0))
df = df.withColumn('energy', F.col('energy').cast(IntegerType()))


+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               0|           0|           0|  2613|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
# ativei o back para tentar agora converter para Pandas
df = backup_energy
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
# ativei o back para tentar agora converter para Pandas
pandas_df = df.toPandas()

In [ ]:
# ativei o back para tentar agora converter para Pandas
print(pandas_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14919 entries, 0 to 14918
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         14919 non-null  object 
 1   song_popularity   14919 non-null  int32  
 2   song_duration_ms  14919 non-null  object 
 3   acousticness      14919 non-null  object 
 4   danceability      14919 non-null  float32
 5   energy            14919 non-null  object 
 6   instrumentalness  14919 non-null  object 
 7   key               14919 non-null  int32  
 8   liveness          14919 non-null  object 
 9   loudness          14919 non-null  object 
 10  audio_mode        14919 non-null  object 
 11  speechiness       14919 non-null  object 
 12  tempo             14919 non-null  object 
 13  time_signature    14919 non-null  object 
 14  audio_valence     14919 non-null  object 
dtypes: float32(1), int32(2), object(12)
memory usage: 1.5+ MB
None


In [ ]:
# fiz um round na string para ter no maximo 5 caracteres
pandas_df["energy"] = pandas_df["energy"].str[:5]

In [ ]:
# apliquei regex para que elimite todos os pontos e deixe apenas o primeiro.
pandas_df["energy"] = pandas_df["energy"].str.replace("\.+", ".", regex=True)

In [ ]:
pandas_df

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Lambada - Original Version 1989,67,207466,0.211,0.756,0.8,723,5,47,-5.158,1,329,118.921,4,967
1,Losing My Religion,28,267733,0.0767,0.672,841,0,9,135,-5.992.000.000.000.000,0,285,125.437,4,0.82
2,Love The Way You Lie,20,158146,0.188,0.437,878,0,10,35.100.000.000.000.000,-3.876,1,396,7.947.399.999.999.990,4,621
3,"Shake, Rattle And Roll",59,150466,0.159,0.671,679,0,5,158,-11.237,1,142,166.726,4,0.82
4,This Old Heart Of Mine (Is Weak For You),19,164960,0.236,0.630,644,0,5,265,-63.210.000.000.000.000,1,29,131.593,4,926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14914,Numbed Up,43,233920,0.114,0.735,621,481,7,235,-4.464,1,576,144.078,4,472
14915,Next Levels,51,229320,0.6509999999999999,0.729,557,123,6,262,-11.264,1,243,92.814,4,897
14916,The Sweet Escape,32,246466,0.191,0.756,0.77,0,1,17.800.000.000.000.000,-3.502,1,343,119.961,4,0.73
14917,Don't Wanna Fall In Love,47,247840,0.0152,0.708,6.809,3,1,373,-11.226,1,422,111.574,4,8.140.000.000.000.000


In [ ]:
# agora enfim e possivel converter para float
pandas_df["energy"] = pandas_df["energy"].astype(float)

In [ ]:
print(pandas_df.info())
# no pandas a coluna acoustincness virou float

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14919 entries, 0 to 14918
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_name         14919 non-null  object 
 1   song_popularity   14919 non-null  int32  
 2   song_duration_ms  14919 non-null  object 
 3   acousticness      14919 non-null  float64
 4   danceability      14919 non-null  float32
 5   energy            14919 non-null  float64
 6   instrumentalness  14919 non-null  object 
 7   key               14919 non-null  int32  
 8   liveness          14919 non-null  object 
 9   loudness          14919 non-null  object 
 10  audio_mode        14919 non-null  object 
 11  speechiness       14919 non-null  object 
 12  tempo             14919 non-null  object 
 13  time_signature    14919 non-null  object 
 14  audio_valence     14919 non-null  object 
dtypes: float32(1), float64(2), int32(2), object(10)
memory usage: 1.5+ MB
None


In [ ]:
# como os numeros de energia variam de 0 a 1
# para todos os numeros estranhos que sao maiores que 1 foram divididos por 1000
def divide_energy(x):
    if x > 1:
        return x / 1000
    else:
        return x

pandas_df["energy"] = pandas_df["energy"].apply(divide_energy)

In [ ]:
# enfim o misterio foi solucionado ! o mesmo processo pode ser aplicado a coluna acousticness agora com o pandas.
pandas_df

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Lambada - Original Version 1989,67,207466,0.211,0.756,0.800000,723,5,47,-5.158,1,329,118.921,4,967
1,Losing My Religion,28,267733,0.0767,0.672,0.841000,0,9,135,-5.992.000.000.000.000,0,285,125.437,4,0.82
2,Love The Way You Lie,20,158146,0.188,0.437,0.878000,0,10,35.100.000.000.000.000,-3.876,1,396,7.947.399.999.999.990,4,621
3,"Shake, Rattle And Roll",59,150466,0.159,0.671,0.679000,0,5,158,-11.237,1,142,166.726,4,0.82
4,This Old Heart Of Mine (Is Weak For You),19,164960,0.236,0.630,0.644000,0,5,265,-63.210.000.000.000.000,1,29,131.593,4,926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14914,Numbed Up,43,233920,0.114,0.735,0.621000,481,7,235,-4.464,1,576,144.078,4,472
14915,Next Levels,51,229320,0.6509999999999999,0.729,0.557000,123,6,262,-11.264,1,243,92.814,4,897
14916,The Sweet Escape,32,246466,0.191,0.756,0.770000,0,1,17.800.000.000.000.000,-3.502,1,343,119.961,4,0.73
14917,Don't Wanna Fall In Love,47,247840,0.0152,0.708,0.006809,3,1,373,-11.226,1,422,111.574,4,8.140.000.000.000.000


In [ ]:
spark_df = spark.createDataFrame(pandas_df)

In [ ]:
spark_df.show()

+--------------------+---------------+----------------+------------+-------------------+--------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|acousticness|       danceability|  energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+------------+-------------------+--------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|       0.211| 0.7559999823570251|     0.8|             723|  5|                  47|              -5.158|         1|        329|             118.921|             4|                 967|
|  Losing My Religion|             2

In [ ]:
df = df.withColumn("key", regexp_replace("key", "[^0-9]+", ""))
df = df.withColumn("key", df["key"].cast(IntegerType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
backup_liveness = df
df = df.withColumn("liveness", regexp_replace("liveness", "[^0-9]+", ""))
df = df.withColumn("liveness", df["liveness"].cast(IntegerType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|     794|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
# a coluna liveness precisa ser convertida para LongType
df = backup_liveness
df = df.withColumn("liveness", regexp_replace("liveness", "[^0-9]+", ""))
df = df.withColumn("liveness", df["liveness"].cast(LongType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
backup_loudness = df
df = df.withColumn("loudness", regexp_replace("loudness", "[^0-9]+", ""))
df = df.withColumn("loudness", df["loudness"].cast(FloatType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
backup_audio_mode= df
df = df.withColumn("audio_mode", regexp_replace("audio_mode", "[^0-9]+", ""))
df = df.withColumn("audio_mode", df["audio_mode"].cast(LongType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
backup_speechiness = df
df = df.withColumn("speechiness", regexp_replace("speechiness", "[^0-9]+", ""))
df = df.withColumn("speechiness", df["speechiness"].cast(LongType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
backup_tempo = df
df = df.withColumn("tempo", regexp_replace("tempo", "[^0-9]+", ""))
df = df.withColumn("tempo", df["tempo"].cast(LongType()))
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|energys|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+
|        0|              0|               1|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|      0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------+



In [ ]:
df.show()

+--------------------+---------------+----------------+------------+------------+----------------+----------------+---+-----------------+-----------------+----------+-----------+-----------------+--------------+--------------------+----------------+
|           song_name|song_popularity|song_duration_ms|acousticness|danceability|          energy|instrumentalness|key|         liveness|         loudness|audio_mode|speechiness|            tempo|time_signature|       audio_valence|         energys|
+--------------------+---------------+----------------+------------+------------+----------------+----------------+---+-----------------+-----------------+----------+-----------+-----------------+--------------+--------------------+----------------+
|Lambada - Origina...|             67|          207466|           0|           0|               8|             723|  5|               47|             5158|         1|        329|           118921|             4|                 967|               8|


In [ ]:
df.show()

+--------------------+---------------+----------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|           song_name|song_popularity|song_duration_ms|        acousticness|      danceability|              energy|instrumentalness|key|            liveness|            loudness|audio_mode|speechiness|               tempo|time_signature|       audio_valence|
+--------------------+---------------+----------------+--------------------+------------------+--------------------+----------------+---+--------------------+--------------------+----------+-----------+--------------------+--------------+--------------------+
|Lambada - Origina...|             67|          207466|               0.211|0.7559999999999999|                 0.8|             723|  5|                  47|              -5.158|         1|        329|             118.9

In [ ]:
#função que remove strings

def remove_strings_inplace(df, colunas):
    # itera sobre as colunas especificadas e aplica a transformação em cada uma
    for column in colunas:
        df = df.withColumn(column, F.regexp_replace(F.col(column), "[^0-9.eE-]", ""))
    return df
#chama a função passando de parâmetro o df e a lista criada

In [ ]:
# tirar a coluna song_name que é formada por sting para nao ser afetada com a função de apagar strings
colunas = df.drop(F.col('song_name')).columns

In [ ]:
# backup para aplicar a transformação
df_backup4 = df

In [ ]:
# aplicar a função de remoção de strings no df
df = remove_strings_inplace(df, colunas)

In [ ]:
# converter as colunas para int e float
# importar as funções para converter


In [ ]:
df_backup5 = df

In [ ]:
df = df_backup5

In [ ]:
df.show()

In [ ]:
#converter os tipos de colunas
df = (df.withColumn('song_popularity', F.col('song_popularity').cast(IntegerType()))
        .withColumn("song_duration_ms", F.col("song_duration_ms").cast(IntegerType()))
        .withColumn("acousticness", F.col("acousticness").cast(FloatType()))
        .withColumn("danceability", F.col("danceability").cast(FloatType()))
        .withColumn("energy", F.col("energy").cast(FloatType()))
        .withColumn("instrumentalness", F.col("instrumentalness").cast(FloatType()))
        .withColumn("key", F.col("key").cast(FloatType()))
        .withColumn("liveness", F.col("liveness").cast(FloatType()))
        .withColumn("loudness", F.col("loudness").cast(FloatType()))
        .withColumn("audio_mode", F.col("audio_mode").cast(FloatType()))
        .withColumn("speechiness", F.col("speechiness").cast(FloatType()))
        .withColumn("tempo", F.col("tempo").cast(IntegerType()))
        .withColumn("time_signature", F.col("time_signature").cast(IntegerType()))
        .withColumn("audio_valence", F.col("audio_valence").cast(FloatType())))

In [ ]:
# criar uma segunda coluna de song duration em segundos
df = df.withColumn('song_duration', F.round(F.col('song_duration_ms') / 60000, 2))

In [ ]:
# dropando a coluna de duração em ms
df = df.drop(F.col('song_duration_ms'))

In [ ]:
# verificando os valores nulos
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|song_name|song_popularity|song_duration_ms|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+
|        0|              0|               0|           0|           0|     0|               0|  0|       0|       0|         0|          0|    0|             0|            0|
+---------+---------------+----------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+



In [ ]:
df.toPandas().song_duration.median()

In [ ]:
df = df.fillna(3.52, subset='song_duration')

In [ ]:
df.toPandas().audio_valence.mode()

0    961
Name: audio_valence, dtype: object

In [ ]:
df = df.fillna(961, subset='audio_valence')

In [ ]:
df.select(F.col('song_name')).distinct().show(truncate=False)

+--------------------------------+
|song_name                       |
+--------------------------------+
|Remedy                          |
|La Gota Fría                    |
|Magic Carpet Ride               |
|My My, Hey Hey (Out of the Blue)|
|You Ain't Seen Nothing Yet      |
|Heaven                          |
|Ginger (feat. Wizkid)           |
|Shake, Rattle And Roll          |
|Hurt You                        |
|Banquet                         |
|Worry About Me                  |
|What Am I To Do                 |
|Debate De 4                     |
|I'm Just Snacking               |
|We Take Care of Our Own         |
|Back for More                   |
|Blue Suede Shoes                |
|Green Green Grass of Home       |
|Hello Walls                     |
|Pancho and Lefty                |
+--------------------------------+
only showing top 20 rows



In [ ]:
df.show()

+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|_c0|           song_name|song_popularity|        acousticness|        danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|
+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|  0|Boulevard of Brok...|             73|0.005520000000000...|          0.496mol/L|                 682|            2.94e-05| 8.0|                 589|              -4.095|         1|                 294| 16

In [ ]:
df.toPandas().isna().any()

_c0                 False
song_name           False
song_popularity     False
acousticness        False
danceability        False
energy              False
instrumentalness    False
key                 False
liveness            False
loudness            False
audio_mode          False
speechiness         False
tempo               False
time_signature      False
audio_valence        True
song_duration       False
dtype: bool

In [ ]:
df.filter((F.col('acousticness').isNull() | F.isnan(F.col('acousticness')) | (F.col('acousticness') == 0))).count()

0

In [ ]:
df.filter((F.col('audio_valence').isNull() | F.isnan(F.col('audio_valence')))).count()

1

In [ ]:
total = df.count() - df.dropDuplicates().count()
print(f'Total de Registros Duplicados = {total}')

Total de Registros Duplicados = 0


In [ ]:
df.select([F.count(F.when(F.col(c) == 'nao_sei', c)).alias(c) for c in df.columns]).show()

+---+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|_c0|song_name|song_popularity|acousticness|danceability|energy|instrumentalness|key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+---+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|  0|        0|              1|           0|           0|     1|               2|  0|       4|       1|         1|          0|    1|             1|            0|            0|
+---+---------+---------------+------------+------------+------+----------------+---+--------+--------+----------+-----------+-----+--------------+-------------+-------------+



In [ ]:
df.filter(F.col('song_popularity').rlike('nao_sei')).show()

+----+-------------------+---------------+------------+------------+------+----------------+-----+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
| _c0|          song_name|song_popularity|acousticness|danceability|energy|instrumentalness|  key|liveness|loudness|audio_mode|speechiness|tempo|time_signature|audio_valence|song_duration|
+----+-------------------+---------------+------------+------------+------+----------------+-----+--------+--------+----------+-----------+-----+--------------+-------------+-------------+
|1757|Party In The U.S.A.|        nao_sei|  0.519mol/L|        0.36|   0.0|              10|177.0|  -8.575|       0|       105|      97.42|    4|           0.7|         null|         3.52|
+----+-------------------+---------------+------------+------------+------+----------------+-----+--------+--------+----------+-----------+-----+--------------+-------------+-------------+



In [ ]:
df.drop_duplicates().count()

18835

In [ ]:
df.show()

+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|_c0|           song_name|song_popularity|        acousticness|        danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|
+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|  0|Boulevard of Brok...|             73|0.005520000000000...|          0.496mol/L|                 682|            2.94e-05| 8.0|                 589|              -4.095|         1|                 294| 16

In [ ]:
df_backup = df

In [ ]:
df_backup.show()

+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|_c0|           song_name|song_popularity|        acousticness|        danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|
+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|  0|Boulevard of Brok...|             73|0.005520000000000...|          0.496mol/L|                 682|            2.94e-05| 8.0|                 589|              -4.095|         1|                 294| 16

In [ ]:
df = df_backup

In [ ]:
df.show()

+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|_c0|           song_name|song_popularity|        acousticness|        danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|
+---+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+
|  0|Boulevard of Brok...|             73|0.005520000000000...|          0.496mol/L|                 682|            2.94e-05| 8.0|                 589|              -4.095|         1|                 294| 16

In [ ]:
from pyspark.sql.types import DecimalType

In [ ]:
df = df.withColumn('acousticness_2', F.split(F.col('acousticness'), 'kg').getItem(0))

In [ ]:
df = df.drop(F.col('acousticness'))

In [ ]:
df.select(F.round(F.col('acousticness_2'), 5)).show()

+------------------------+
|round(acousticness_2, 5)|
+------------------------+
|                 0.00552|
|                  0.0103|
|                 0.00817|
|                  0.0264|
|                  9.5E-4|
|                 0.00895|
|                  5.0E-4|
|                 0.00148|
|                 0.00108|
|                 0.00172|
|                  0.0424|
|                  0.0046|
|                 0.00434|
|                  0.0179|
|                  3.5E-4|
|                  4.2E-4|
|                 0.00136|
|                 0.00701|
|                  0.0938|
|                 0.00664|
+------------------------+
only showing top 20 rows



In [ ]:
 df.withColumn("acousticness_float", F.col("acousticness_2").cast("float")).show()

+---+--------------------+---------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+------------------+
|_c0|           song_name|song_popularity|      danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|      acousticness_2|acousticness_float|
+---+--------------------+---------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+------------------+
|  0|Boulevard of Brok...|             73|             0.496|                 682|            2.94e-05| 8.0|                 589|              -4.095|       

In [ ]:
from pyspark.sql.functions import col, regexp_extract

In [ ]:
df_backup2= df

In [ ]:
#df.withColumn("acousticness_float", (F.regexp_extract(F.col("acousticness_float"), "([-+]?[0-9]*\.?[0-9]+([eE][-+]?[0-9]+)?)", 1)).cast("float")).show()

#df.withColumn("acousticness_float", F.regexp_extract(col("acousticness_float"), "([0-9]*\.?[0-9][0-9]([eE][-+]?[0-9]+)?(\.[0-9]+)?)", 1).cast("float")).show()
df.withColumn("instrumentalness", F.regexp_extract(col("instrumentalness"), '([0-9]*\.?[0-9][0-9]([eE][-+]?[0-9]+)?(\.[0-9]+)?)', 1).cast("float")).show()


+---+--------------------+---------------+------------------+--------------------+----------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+
|_c0|           song_name|song_popularity|      danceability|              energy|instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|      acousticness_2|
+---+--------------------+---------------+------------------+--------------------+----------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+
|  0|Boulevard of Brok...|             73|             0.496|                 682|         2.94E-5| 8.0|                 589|              -4.095|         1|                 294| 167.06|             4|                 474|    

In [ ]:
from pyspark.sql.functions import round

In [ ]:
df_backup2=df

In [ ]:
df = df.withColumn("instrumentalness_2", round(col("instrumentalness").cast("decimal(38,20)").cast("float"), 6))

In [ ]:
from pyspark.sql.functions import col, regexp_replace

In [ ]:
df = df.withColumn("instrumentalness_2", regexp_replace(col("instrumentalness_2"), "[^\\d\\.]","").cast("float"))

In [ ]:
df.show()

+---+--------------------+---------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+------------------+
|_c0|           song_name|song_popularity|      danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|      acousticness_2|instrumentalness_2|
+---+--------------------+---------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+------------------+
|  0|Boulevard of Brok...|             73|             0.496|                 682|            2.94e-05| 8.0|                 589|              -4.095|       

In [ ]:
df = df.fillna(0.0, subset='instrumentalness_2')

In [ ]:
df = df_backup

In [ ]:
df = df.withColumn('danceability', F.split(F.col('danceability'), 'mol/L').getItem(0))

In [ ]:
df.show()

+---+--------------------+---------------+--------------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+
|_c0|           song_name|song_popularity|        acousticness|      danceability|              energy|    instrumentalness| key|            liveness|            loudness|audio_mode|         speechiness|  tempo|time_signature|       audio_valence|song_duration|      acousticness_2|
+---+--------------------+---------------+--------------------+------------------+--------------------+--------------------+----+--------------------+--------------------+----------+--------------------+-------+--------------+--------------------+-------------+--------------------+
|  0|Boulevard of Brok...|             73|0.005520000000000...|             0.496|                 682|            2.94e-05| 8.0|                 589| 